# Welcome to MoSeq2-Notebook

### Run all of the MoSeq tools in a containerized notebook.

## Setup

To begin, run the following jupyter notebook to ensure MoSeq2 is installed and running smoothely on your machine.
[SETUP NOTEBOOK](http://localhost:8889/notebooks/MoSeq2_Step_0.ipynb)

Next, input your directory path containing your recorded sessions for extraction and analysis. This is will also serve as your current working directory later on for all MoSeq operations.

In [2]:
import os
os.listdir("Insert Absolute Session Path here")

['.DS_Store', 'session-reg-0', 'session-teth-0']

If the files are correct, confirm the path to generate an extraction configuration file to aid in your extraction.

In [6]:
# Putting this off for now

/Users/aymanzeine/Desktop/Playground/recordings


## Raw Data Extraction Step

To extract all the sessions in your inputted directory, simply run the default command below.
Otherwise, run the command with the associated session index above (e.g. ```moseq2-extract extract 1,2,3```)

In [4]:
%%bash
moseq2-extract extract depth.dat

/Users/aymanzeine/Desktop/Moseq_local/moseq2-app


## Principal Component Analysis (PCA) Step

Once all your data is extracted and saved in your desired proc/ directory, you are now able to perform the PCA step.

### Training

To train your PCA on your extracted data results, run the following command. It will recursively search within your current directory structure for your extracted results_xx.h5 files.

In [2]:
%%bash
moseq2-pca train-pca

### Computing Principal Component Scores
Once your PCA model has been trained, you can now apply your model using your extracted data amd computed principal components. To compute your PC Scores, run the following command:

In [3]:
moseq2-pca apply-pca

### Computing Model-free Syllable Changepoints
To measure block duration distances between detected syllables using your PCA model or computed scores, you can run the following command:

In [4]:
%%bash
moseq2-pca compute-changepoints

## Train ARHMM

Once you have computed your PCA Scores, you can now use this data as your input to train your Auto-Regressive Heuristic Markov Model (ARHMM).
If you have multiple groups (for example, a control and experimental group) that you would like to model separately using the same model, use the ```--separate-trans``` flag in the command below.

### Single Group Training

In [5]:
%%bash
moseq2-model learn-model pca_scores.h5 model.p

### Multiple Group Training
In order to model multiple groups separately in your model you must generate an index file to point to all your relevant paths, as well as indicate use the separate transition graphs flag:

In [ ]:
%%bash
moseq2-viz generate-index
moseq2-viz add-group 
moseq2-model learn-model -k 123456 --separate-trans pca_scores.h5 model.p

## Visualize Results

Now that you have a trained ARHMM, you can use the moseq2-viz module to produce crowd videos and a number of statistical analysis plots.

### Setup
Ensure that you have a `moseq2-index.yaml` file generated using this command:

In [6]:
%%bash
moseq2-viz generate-index

### Make Crowd Videos
This tool allows you to create videos containing many overlayed clips of the mouse performing the same specified syllable at the moment a red dot appears on their body. The videos are sorted by most frequently expressed syllable to least.
To create the crowd videos, run the following command:

In [7]:
%%bash
moseq2-viz make-crowd-movies

### Compute Usage Plots
Use this command to compute the model-detected syllables usages sorted in descending order of usage.

In [ ]:
%%bash
moseq2-viz plot-usages

### Compute Scalar Summary and Tracking Plots
Use the following command to compute some scalar summary information about your modeled groups, such as average velocity, height, etc.
This command will also generate a tracking summary plot; depicting the path traveled by the mouse in your recordings.

In [8]:
%%bash
moseq2-viz plot-scalars

### Compute Syllable Transition Graph
Use the following command to generate a syllable transition graph. The graph will be comprised of nodes labelled by syllable, and edges depicting a probable transition, with edge thickness depicting the weight of the transition edge.

For multiple groups, there will be a transition graph for each group, as well as a unified graph with different colors to identify the groups.

In [ ]:
%%bash
moseq2-viz plot-transition-graph